In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import functools
import diffusion
import matplotlib.pyplot as plt

In [ ]:
figure = diffusion.plot_slope_distribution(output_path=None)
plt.show(figure)
plt.close(figure)